In [1]:
import numpy as np

from keras.models import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from models.unet_se import *
from models.unet import *
from models.resnet_fcn import *
from models.resnet_se_fcn import *
from models.resnet_fcn import *
from models.vgg19_fcn import *
from models.vgg19_se_fcn import *
from models.unet_resnet import *

from datahandler import DataHandler
from data_loader import *
from params import *
import os
import cv2
import skimage.io as io
from tqdm import tqdm

from medpy.io import save

from math import ceil, floor
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, jaccard_similarity_score

from scipy.ndimage import _ni_support
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion,\
     generate_binary_structure

import warnings
warnings.filterwarnings("ignore")

plt.gray()

Using TensorFlow backend.


In [2]:
def destiny_directory(dice_score):
    pre = './data/eval/unet_se/'
    if dice_score >= 98:
        return pre + 'dice_98_100/'
    elif dice_score >= 96:
        return pre + 'dice_96_98/'
    elif dice_score >= 94:
        return pre + 'dice_94_96/'
    elif dice_score >= 92:
        return pre + 'dice_92_94/'
    elif dice_score >= 90:
        return pre + 'dice_90_92/'
    elif dice_score >= 88:
        return pre + 'dice_88_90/'
    elif dice_score >= 85:
        return pre + 'dice_85_88'
    elif dice_score >= 80:
        return pre + 'dice_80_85/'
    elif dice_score >= 70:
        return pre + 'dice_70_80/'
    elif dice_score >= 60:
        return pre + 'dice_60_70/'
    else:
        return pre + 'dice_less_60'

In [3]:
def getGenerator(images, bs=1):
    image_datagen = ImageDataGenerator(rescale=1./255)
    image_datagen.fit(images, augment = True)
    image_generator = image_datagen.flow(x = images, batch_size=bs,
            shuffle = False)

    return image_generator


In [4]:
def getDiceScore(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)


In [ ]:
def getModel(model_type):
    if model_type == 'unet':
        print('working with unet')
        model = getUnet()
    elif model_type == 'resnetFCN':
        print('working with resnetFCN')
        model = getResnet50FCN()
    elif model_type == 'resnetSEFCN':
        print('working with resnetSEFCN')
        model = getResnetSE50FCN()
    elif model_type == 'vgg19FCN':
        print('working with vgg19FCN')
        model = getVGG19FCN()
    elif model_type == 'vgg19SEFCN':
        print('working with vgg19SEFCN')
        model = getVGG19SEFCN()
    elif model_type == 'UnetResNet18':
        print('working with UnetResNet18')
        model = getUnetResnet18()
    elif model_type == 'UnetResNet18SE':
        print('working with UnetResNet18SE')
        model = getUnetResnet18(se_version = True)
    elif model_type == 'unet_se':
        print('working with unet_se')
        model = getSEUnet()

    return model

In [5]:
def hd(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

def hd95(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity)
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity)
    hd95 = np.percentile(np.hstack((hd1, hd2)), 95)
    return hd95

def __surface_distances(result, reference, voxelspacing=None, connectivity=1):
    result = np.atleast_1d(result.astype(np.bool))
    reference = np.atleast_1d(reference.astype(np.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = np.asarray(voxelspacing, dtype=np.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()

    footprint = generate_binary_structure(result.ndim, connectivity)

    if 0 == np.count_nonzero(result):
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == np.count_nonzero(reference):
        raise RuntimeError('The second supplied array does not contain any binary object.')

    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)

    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]

    return sds

In [6]:
image_files, mask_files = load_data_files('data/kfold_data/')
print(len(image_files))
print(len(mask_files))
skf = getKFolds(image_files, mask_files, n=10)

kfold_indices = []
for train_index, val_index in skf.split(image_files, mask_files):
    kfold_indices.append({'train': train_index, 'val': val_index})

291
291


In [7]:
def predictMask(model, image):  
    image_gen = getGenerator(image)
    return model.predict_generator(image_gen, steps=len(image))

In [8]:
def prepareForSaving(image):
    image = np.squeeze(image)
    image = np.swapaxes(image, -1, 0)
    
    return image

def predictAll(model, data, num_data=0):
    dice_scores = []
    hd_scores = []
    hd95_scores = []

    for image_file, mask_file in tqdm(data, total=num_data):
        
        fname = image_file[image_file.rindex('/')+1 : image_file.index('.')]

        image, hdr = dh.getImageData(image_file)
        gt_mask, _ = dh.getImageData(mask_file, is_mask=True)

        assert image.shape == gt_mask.shape
        
        if image.shape[1] != 256:
            continue
            
        pred_mask = predictMask(model, image)
        pred_mask[pred_mask>=0.5] = 1
        pred_mask[pred_mask<0.5] = 0
            
        dice_score = getDiceScore(gt_mask, pred_mask)
        
        if dice_score == 0:
            dice_scores.append(dice_score)
            hd_scores.append(200)
            hd95_scores.append(200)
            
            continue
        
        dice_scores.append(dice_score)
        
        hd_score = hd(gt_mask, pred_mask)
        hd_scores.append(hd_score)
        
        hd95_score = hd95(gt_mask, pred_mask)
        hd95_scores.append(hd95_score)
        
        '''int_dice_score = floor(dice_score * 100)
        save_path = destiny_directory(int_dice_score)
        
        pred_mask = prepareForSaving(pred_mask)
        image = prepareForSaving(image)
        gt_mask = prepareForSaving(gt_mask)
        
        save(pred_mask, os.path.join(save_path, fname + '_' + unet_type + '_' 
            + str(int_dice_score) + '.nii'), hdr)
        save(image, os.path.join(save_path, fname + '_img.nii'), hdr)
        save(gt_mask, os.path.join(save_path, fname + '_mask.nii'), hdr)'''

    return dice_scores, hd_scores, hd95_scores

In [9]:
#Get data and generators

unet_type = 'unet_se'
dh = DataHandler()
all_dice = []
all_hd = []
all_hd95 = []

for i in range(len(kfold_indices)):
    exp_name = 'kfold_%s_dice_DA_K%d'%(unet_type, i)

    #get parameters
    params = getParams(exp_name, unet_type=unet_type)
    
    val_img_files = np.take(image_files, kfold_indices[i]['val'])
    val_mask_files = np.take(mask_files, kfold_indices[i]['val'])
    
    
    model = getModel(unet_type)
    
    print('loading weights from %s'%params['checkpoint']['name'])
    model.load_weights(params['checkpoint']['name'])
        
    data = zip(val_img_files, val_mask_files)
    
    dice_score, hd_score, hd95_score = predictAll(model, data, num_data=len(val_mask_files))
    
    print('Finished K%d'%i)
    
    all_dice += dice_score
    all_hd += hd_score
    all_hd95 += hd95_score

print('dice')
for i in range(len(all_dice)):
    print(all_dice[i])
print()

print('hd')
for i in range(len(all_hd)):
    print(all_hd[i])
print()

print('hd95')
for i in range(len(all_hd95)):
    print(all_hd95[i])
print()

print('Final results for %s'%unet_type)
print('dice %f'%np.mean(all_dice))
print('hd %f'%np.mean(all_hd))
print('hd95 %f'%np.mean(all_hd95))


Working with unetSE
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_se_dice_DA_K0/kfold_unet_se_dice_DA_K0_weights.h5


100%|██████████| 30/30 [02:03<00:00,  3.84s/it]


Finished K0
Working with unetSE
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_se_dice_DA_K1/kfold_unet_se_dice_DA_K1_weights.h5


100%|██████████| 29/29 [02:06<00:00,  4.22s/it]


Finished K1
Working with unetSE
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_se_dice_DA_K2/kfold_unet_se_dice_DA_K2_weights.h5


...

Finished K7
Working with unetSE
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_se_dice_DA_K8/kfold_unet_se_dice_DA_K8_weights.h5


100%|██████████| 29/29 [02:10<00:00,  5.40s/it]


Finished K8
Working with unetSE
loading weights from ./logs/unet_se/kfold_unet_se/kfold_unet_se_dice_DA_K9/kfold_unet_se_dice_DA_K9_weights.h5


100%|██████████| 29/29 [01:54<00:00,  3.65s/it]

Finished K9
dice
0.9539159459309936
0.9546579028151466
0.9097918597792197
0.9404981793234471
0.948295141908802
0.9733055630634211
0.9739000393545849
0.9407617105792234
0.9678839214883435
0.9645576465498594
0.9651333891801499
0.9571235026365292
0.9508122177621676
0.9762698341204783
0.9738496791478405
0.9395112497855749
0.9488351180877693
0.9155031235222381
0.0
0.9710668220889128
0.16597331028148468
0.9487803215654753
0.9753705909678368
0.9690470079896333
0.951474188322116
0.8607907576918425
0.945740606303352
0.9371511616981025
0.9437098758412504
0.9520877309119068
0.9397382920110192
0.9658651824861358
0.9321411696289027
0.9742243570052221
0.9484547323820799
0.9626644616334691
0.9490419795879069
0.9763188971633769
0.5519691119691119
0.9577237529819577
0.9618965949632314
0.9682270271156699
0.955664293499687
0.9210745195009553
0.9470717974562816
0.922064545662623
0.9703354588300528
0.9682107935445173
0.9655426570849456
0.9527736806482028
0.968316282354128
0.9499290908542289
0.9552051633615